In [67]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [151]:
from implementations import *
from preprocessing import *
from visualization import *
from featurization import *
from hyperoptim import *
from genetic_algorithm import *
from proj1_helpers import *

# Training and evaluation without feature expansion

#### 1. Load data

In [68]:
DATA_TRAIN_PATH = '../data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

#### 2. Data preprocessing
1. Splitting data by pri_jet_num column

In [71]:
tX_split, y_split, idx_jet_num = split_dataset_by_jet_num(tX, y, remove_cols=True)
tX_0 = tX_split[0]
tX_1 = tX_split[1]
tX_2 = tX_split[2]

After splitting the dataset based on the PRI_jet_num only the column DER_mass_MMC has -999 values<br>

2. Filling NaNs and removing outliers

In [72]:
tX_0 = clean_data(tX_0, fill_outliers=False)
tX_1 = clean_data(tX_1, fill_outliers=False)
tX_2 = clean_data(tX_2, fill_outliers= False)

y0 = y_split[0]
y1 = y_split[1]
y2 = y_split[2] 

In [73]:
# # Tukey's fences
# tX_0, y0 = remove_outliers(tX_0, y0, k=3, sigma_rule=False)
# tX_1, y1 = remove_outliers(tX_1, y1, k=3, sigma_rule=False)
# tX_2, y2 = remove_outliers(tX_2, y2, k=3, sigma_rule=False)

3. Standardization of the data

In [74]:
tX_0, tX_0_means, tX_0_stds  = standardize_data(tX_0)
tX_1, tX_1_means, tX_1_stds  = standardize_data(tX_1)
tX_2, tX_2_means, tX_2_stds  = standardize_data(tX_2)

4. Spliting into train and test sets

In [75]:
ratio = 0.7
train0, test0 = split_data(tX_0, y0, ratio)
train1, test1 = split_data(tX_1, y1, ratio)
train2, test2 = split_data(tX_2, y2, ratio)

#### 3. Training and evaluation

In [111]:
def print_fscore(keep, keep_obj):
    '''
    Callbasck funcition used in radnom search hyperparameters optimization
    For more details see documentation for hp_random_search
    '''
    print("Accuaracy Test: " + str(keep_obj['test_perf']['accuracy']))

def print_performance(all_weights, tX, ys):
    '''
    Calculates and prints performance parameters (recall, precision, accuracy, fscore, TP,TN ,FP ,FN)
    Node that correspondence between all_weights[i], tX[i] and ys[i] is obligatory 
    
    Parameters:
    ----------
    all_weights: list or touple
        Array-like structure of weight vectors
    tX: list or touple
     Array-like structure of sets of features
    ys: list or touple
      Array-like structure of sets of labels
    
    Returns:
    --------
    void
    '''
    labels = np.array([])
    y = np.array([])
    for i in range(len(all_weights)):
        labels = np.concatenate([labels ,predict_labels(all_weights[i],tX[i])])
        y = np.concatenate([y, ys[i]])
    TP = ((y==1)&(labels==1)).sum()
    FP = ((y==1)&(labels==-1)).sum()
    TN = ((y==-1)&(labels==-1)).sum()
    FN = ((y==-1)&(labels==1)).sum()
    precision = TP/(TP+FP)
    recall = TP / (TP + FN)
    accuracy = (TP + TN)/(TP + TN + FP + FN)
    fscore = 2 * (precision*recall)/(precision + recall)
    perf = {
        'TP': TP,
        'FP': FP,
        'TN': TN,
        'FN': FN,
        'precision': precision,
        'recall': recall,
        'accuracy': accuracy,
        'fscore': fscore
    }
    print( perf)

1. Least squares

In [113]:
def least_squares_training(train):
    #LEAST  SQUARES
    y_train =train[1]
    tX_train = train[0]
   
    weights, loss = least_squares(y_train,tX_train)
    return weights


weights0_ls = least_squares_training(train0)
weights1_ls = least_squares_training(train1)
weights2_ls = least_squares_training(train2)

print('Least squares with train/test split')
print_performance((weights0_ls,weights1_ls, weights2_ls), (test0[0], test1[0], test2[0]), (test0[1], test1[1], test2[1]))

Least squares with train/test split
{'TP': 20743, 'FP': 4923, 'TN': 32350, 'FN': 16985, 'precision': 0.8081898231122886, 'recall': 0.5498038592027141, 'accuracy': 0.7078972280369595, 'fscore': 0.6544152443448906}


2. Ridge regression

In [117]:
def ridge_resgression_training(train, lambda_):
    y_train =train[1]
    tX_train = train[0]

    initial_w = np.random.randn(tX_train.shape[1])
    weights, loss = ridge_regression(y_train, tX_train, lambda_)
    return weights


lambda_ = 0.1
weights0_rr = ridge_resgression_training(train0, lambda_)
weights1_rr= ridge_resgression_training(train1, lambda_)
weights2_rr =ridge_resgression_training(train2, lambda_)

print('Ridge regression with train/test split')
print_performance((weights0_rr,weights1_rr, weights2_rr), (test0[0], test1[0], test2[0]), (test0[1], test1[1], test2[1]))

Ridge regression with train/test split
{'TP': 20223, 'FP': 5443, 'TN': 32521, 'FN': 16814, 'precision': 0.78792955661186, 'recall': 0.5460215460215461, 'accuracy': 0.7032439567472434, 'fscore': 0.64504090713363}


3. Regularized logistical regression

In [119]:
def reg_logistical_resgression_training( train, lambda_, gamma, max_iters):
    y_train =train[1]
    tX_train = train[0]
    initial_w = np.random.randn(tX_train.shape[1])
    weights, loss = reg_logistic_regression(y_train, tX_train,lambda_, initial_w, max_iters, gamma)
    return weights


max_iters = 1000
lambda_ = 0.1
gamma = 0.0001

weights0_rlg = reg_logistical_resgression_training(train0, lambda_, gamma, max_iters)
weights1_rlg = reg_logistical_resgression_training(train1, lambda_, gamma, max_iters)
weights2_rlg = reg_logistical_resgression_training(train2, lambda_, gamma, max_iters)


print('Regularized logistical regression with train/test split')
print_performance((weights0_rlg,weights1_rlg, weights2_rlg), (test0[0], test1[0], test2[0]), (test0[1], test1[1], test2[1]))

Regularized logistical regression with train/test split
{'TP': 19995, 'FP': 5671, 'TN': 33567, 'FN': 15768, 'precision': 0.7790462089924414, 'recall': 0.5590973911584599, 'accuracy': 0.7141504779936267, 'fscore': 0.6509954581712221}


4. Logistical regression

In [121]:
def logistical_resgression_training( train, gamma, max_iters):
    y_train =train[1]
    tX_train = train[0]
    initial_w = np.random.randn(tX_train.shape[1])
    weights, loss = logistic_regression(y_train, tX_train, initial_w, max_iters, gamma)
    return weights


max_iters = 1000
gamma = 0.0001

weights0_lg = logistical_resgression_training(train0, gamma, max_iters)
weights1_lg = logistical_resgression_training(train1, gamma, max_iters)
weights2_lg = logistical_resgression_training(train2, gamma, max_iters)


print('Logistical regression with train/test split')
print_performance((weights0_lg,weights1_lg, weights2_lg), (test0[0], test1[0], test2[0]), (test0[1], test1[1], test2[1]))

Logistical regression with train/test split
{'TP': 19996, 'FP': 5670, 'TN': 33571, 'FN': 15764, 'precision': 0.7790851710434037, 'recall': 0.55917225950783, 'accuracy': 0.7142171437714164, 'fscore': 0.6510598118060756}


5. Gradient Descent

In [125]:
def least_squaresGD_training( train, gamma, max_iters):
    y_train =train[1]
    tX_train = train[0]
    initial_w = np.random.randn(tX_train.shape[1])
    weights, loss = least_squares_GD(y_train, tX_train, initial_w, max_iters, gamma)
    return weights


max_iters = 1000
gamma = 0.01

weights0_lsGD = least_squaresGD_training(train0, gamma, max_iters)
weights1_lsGD = least_squaresGD_training(train1, gamma, max_iters)
weights2_lsGD = least_squaresGD_training(train2, gamma, max_iters)


print('Gradient Descent least squared with train/test split')
print_performance((weights0_lsGD,weights1_lsGD, weights2_lsGD), (test0[0], test1[0], test2[0]), (test0[1], test1[1], test2[1]))

Gradient Descent least squared with train/test split
{'TP': 19950, 'FP': 5716, 'TN': 31622, 'FN': 17713, 'precision': 0.777292916699135, 'recall': 0.529697581180469, 'accuracy': 0.6876174984333542, 'fscore': 0.6300431082126671}


6. Stochastic Gradient Descent

In [124]:
def least_squaresSGD_training( train, gamma, max_iters):
    y_train =train[1]
    tX_train = train[0]
    initial_w = np.random.randn(tX_train.shape[1])
    weights, loss = least_squares_SGD(y_train, tX_train, initial_w, max_iters, gamma)
    return weights


max_iters = 1000
gamma = 0.01

weights0_lsSGD = least_squaresSGD_training(train0, gamma, max_iters)
weights1_lsSGD = least_squaresSGD_training(train1, gamma, max_iters)
weights2_lsSGD = least_squaresSGD_training(train2, gamma, max_iters)


print('Gradient Descent least squared with train/test split')
print_performance((weights0_lsSGD,weights1_lsSGD, weights2_lsSGD), (test0[0], test1[0], test2[0]), (test0[1], test1[1], test2[1]))


Gradient Descent least squared with train/test split
{'TP': 19703, 'FP': 5963, 'TN': 31847, 'FN': 17488, 'precision': 0.7676692901114315, 'recall': 0.5297787099029335, 'accuracy': 0.6873241690110798, 'fscore': 0.6269150611705936}


# Hyperparameters optimization with feature expansion

#### 1. Load data

In [163]:
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

#### 2. Training data preprocessing

In [164]:
#extract sample for testing
ratio = 0.1
ratio = int(0.1*tX.shape[0])
indices = np.random.permutation(tX.shape[0])

tX_test = tX[indices[:ratio]]
y_test  = y[indices[:ratio]]

tX = tX[indices[ratio:]]
y  = y[indices[ratio:]]

In [165]:
#split data by pri_jet_num
tX_split, y_split, idx_jet_num = split_dataset_by_jet_num(tX, y, remove_cols=True)
tX_0 = tX_split[0]
tX_1 = tX_split[1]
tX_2 = tX_split[2]

y0 = y_split[0]
y1 = y_split[1]
y2 = y_split[2] 

In [166]:
#clean data (fill NaNs, transform outliers)
tX_0 = clean_data(tX_0, fill_outliers=False)
tX_1 = clean_data(tX_1, fill_outliers=False)
tX_2 = clean_data(tX_2, fill_outliers= False)

In [167]:
#standardize data
tX_0, tX_0_means, tX_0_stds  = standardize_data(tX_0)
tX_1, tX_1_means, tX_1_stds  = standardize_data(tX_1)
tX_2, tX_2_means, tX_2_stds  = standardize_data(tX_2)

In [168]:
#split on train and validation sets 
ratio = 0.7
train0, validate0 = split_data(tX_0, y0, ratio)
train1, validate1 = split_data(tX_1, y1, ratio)
train2, validate2 = split_data(tX_2, y2, ratio)

#### 3. Test data preprocessing

In [169]:
#split data by pri_jet_num
tX_split_test, y_split_test, idx_jet_num = split_dataset_by_jet_num(tX_test,y_test, remove_cols=True)
tX_0_test = tX_split_test[0]
tX_1_test = tX_split_test[1]
tX_2_test = tX_split_test[2]

#clean NaNs
tX_0_test = clean_data(tX_0_test)
tX_1_test = clean_data(tX_1_test)
tX_2_test = clean_data(tX_2_test)

#means and std are learned on training data
tX_0_test, _ , _  = standardize_data(tX_0_test, tX_0_means, tX_0_stds)
tX_1_test, _ , _   = standardize_data(tX_1_test, tX_1_means, tX_1_stds)
tX_2_test, _ , _   = standardize_data(tX_2_test, tX_2_means, tX_2_stds)

## 1. Random search training

In [ ]:
#training models using random search
#note that np.random.seed is preset, so random search will always return same best models

#training Model 0
best10_0=hp_random_search(train0[0], train0[1], validate0[0], validate0[1], 
                          gamma = 1e-4, model_num=2, keep_callback=print_fscore,search_max_iters= 200, polynom_exp=10)
weights0_random_search = best10_0[0]['weights']
expansion0_random_search = best10_0[0]['expansion']

#training Model 1
best10_1=hp_random_search(train1[0], train1[1], validate1[0], validate1[1],
                          gamma = 1e-4, model_num=2, keep_callback=print_fscore,search_max_iters= 200, polynom_exp=10)
weights1_random_search = best10_1[0]['weights']
expansion1_random_search = best10_1[0]['expansion']

#training Model 2
best10_2=hp_random_search(train2[0], train2[1], validate2[0], validate2[1],
                          gamma = 1e-4, model_num=2, keep_callback=print_fscore,search_max_iters= 200, polynom_exp=10)
weights2_random_search = best10_2[0]['weights']
expansion2_random_search = best10_2[0]['expansion']

#### 4. Performance evaluation

In [178]:
#feature expansion
tX_0_test_random_search = polynomial_feature_expansion(tX_0_test, expansion0_random_search)
tX_1_test_random_search = polynomial_feature_expansion(tX_1_test, expansion1_random_search)
tX_2_test_random_search = polynomial_feature_expansion(tX_2_test, expansion2_random_search)

#evaluation on test set
print('Random search: Model 0:')
print(calculate_performance(weights0_random_search, tX_0_test_random_search, y_split_test[0]))

print('\n\n\nRandom search: Model 1:')
print(calculate_performance(weights1_random_search, tX_1_test_random_search, y_split_test[1]))

print('\n\n\nRandom search: Model 1:')
print(calculate_performance(weights2_random_search, tX_2_test_random_search, y_split_test[2]))


print('\n\n\nOverall performance')
print_performance((weights0_random_search,weights1_random_search, weights2_random_search),
                  (tX_0_test_random_search, tX_1_test_random_search, tX_2_test_random_search),
                  ( y_split_test[0],  y_split_test[1],  y_split_test[2]))

Random search: Model 0:
{'TP': 1562, 'FP': 967, 'TN': 6915, 'FN': 568, 'precision': 0.6176354290233294, 'recall': 0.7333333333333333, 'accuracy': 0.8466839792249301, 'fscore': 0.670530156685984}



Random search: Model 1:
{'TP': 1878, 'FP': 847, 'TN': 4293, 'FN': 717, 'precision': 0.6891743119266055, 'recall': 0.7236994219653179, 'accuracy': 0.7978021978021979, 'fscore': 0.7060150375939849}



Random search: Model 1:
{'TP': 2616, 'FP': 621, 'TN': 3342, 'FN': 674, 'precision': 0.8081556997219648, 'recall': 0.7951367781155015, 'accuracy': 0.8214531917827106, 'fscore': 0.8015933813390532}



Overall performance
{'TP': 6056, 'FP': 2435, 'TN': 14550, 'FN': 1959, 'precision': 0.7132257684607232, 'recall': 0.7555832813474734, 'accuracy': 0.82424, 'fscore': 0.7337937719617109}


## 2. Genetic algorithm training

In [ ]:
#training Model 0
best0 = genetic_algo(train0, test0, model_num_= 2, max_degree_=5)
weights0_genetic_algo = best0['weights']
expansion0_genetic_algo = best0['expansion']

#training Model 1
best1 = genetic_algo(train1, test1, model_num_= 2, max_degree_=5)
weights1_genetic_algo = best1['weights']
expansion1_genetic_algo = best1['expansion']

#training Model 2
best2 = genetic_algo(train2, test2, model_num_= 2, max_degree_=5)
weights2_genetic_algo = best2['weights']
expansion2_genetic_algo = best2['expansion']

#### 4. Performance evaluation

In [180]:
#feature expansion
tX_0_test_genetic_algo = polynomial_feature_expansion(tX_0_test, expansion0_genetic_algo)
tX_1_test_genetic_algo = polynomial_feature_expansion(tX_1_test, expansion1_genetic_algo)
tX_2_test_genetic_algo = polynomial_feature_expansion(tX_2_test, expansion2_genetic_algo)


#evaluation on test set
print('Genetic algorithm: Model 0:')
print(calculate_performance(weights0_genetic_algo, tX_0_test_genetic_algo, y_split_test[0]))

print('\n\n\nGenetic algorithm: Model 1:')
print(calculate_performance(weights1_genetic_algo, tX_1_test_genetic_algo, y_split_test[1]))

print('\n\n\nGenetic algorithm: Model 2:')
print(calculate_performance(weights2_genetic_algo, tX_2_test_genetic_algo, y_split_test[2]))

print('\n\n\nOverall performance')
print_performance((weights0_genetic_algo,weights1_genetic_algo, weights2_genetic_algo),
                  (tX_0_test_genetic_algo, tX_1_test_genetic_algo, tX_2_test_genetic_algo),
                  ( y_split_test[0],  y_split_test[1],  y_split_test[2]))

Genetic algorithm: Model 0:
{'TP': 1516, 'FP': 1013, 'TN': 6861, 'FN': 622, 'precision': 0.5994464215104784, 'recall': 0.7090739008419084, 'accuracy': 0.8366959648421893, 'fscore': 0.6496678808656525}



Genetic algorithm: Model 1:
{'TP': 1663, 'FP': 1062, 'TN': 4275, 'FN': 735, 'precision': 0.6102752293577982, 'recall': 0.6934945788156798, 'accuracy': 0.7676793794440854, 'fscore': 0.649228967401913}



Genetic algorithm: Model 2:
{'TP': 2484, 'FP': 753, 'TN': 3325, 'FN': 691, 'precision': 0.7673772011121409, 'recall': 0.7823622047244094, 'accuracy': 0.8009099682889839, 'fscore': 0.7747972551466001}



Overall performance
{'TP': 5663, 'FP': 2828, 'TN': 14461, 'FN': 2048, 'precision': 0.6669414674361088, 'recall': 0.7344053948904163, 'accuracy': 0.80496, 'fscore': 0.6990495000617208}


In [27]:
tX = expand_one_hot(tX, 22, 4)

In [28]:
tX = clean_data(tX,replace_with_zeros)
tX_standardized = standardize_data(tX)

In [29]:
ratio = 0.7
train_data , test_data  = split_data(tX, y, ratio)

tX_train = train_data[0]
y_train = train_data[1]

tX_test = test_data[0]
y_test = test_data[1]


In [ ]:
least_squares_training(test_data,train_data)

In [ ]:
max_iters = 1000
lambda_ = 0.1
gamma = 0.00001
reg_logistical_resgression_training(test_data,train_data, lambda_, gamma, max_iters)

In [ ]:
lambda_ = 0.1
ridge_resgression_training(test_data,train_data, lambda_)
print(tX.shape)

In [ ]:
def print_fscore(keep, keep_obj):
    print("FScore Test: " + str(keep_obj['test_perf']['fscore']))
    
best=hp_random_search(tX_train, y_train, tX_test,y_test, gamma = 1e-4, model_num=1, keep_callback=print_fscore,search_max_iters= 200,  expansion_mask = [ i<22 and i>25  for i in range(tX_train.shape[1])],polynom_exp=10)

### 

# Generate predictions and save ouput in csv format for submission:

#### 1. Read data

In [141]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

#### 2. Best models extracted from multiple simulations

In [127]:
# model 0
weights0 = np.array([ 6.19431299e-02, -2.13454520e-01, -3.46570557e-01, -1.23822041e+03,
          4.29467861e-01,  1.23813972e+03, -1.81487127e+01,  1.62186883e+00,
          1.39336416e+01, -5.43522262e-04,  9.98507322e+00,  1.31353813e-02,
         -2.98862771e-02, -1.04175169e-01, -1.05349952e-02,  2.92969000e-02,
         -1.89101725e-01,  1.90867517e-01, -2.05318085e-01, -2.18270585e-02,
          1.15732175e-01,  3.81785587e-01, -1.68797404e-01, -7.58202766e-01,
          1.28811172e-03, -1.43703538e-01, -5.14037178e-02, -3.92820429e-02,
          5.96254127e-03,  1.92115081e-03,  2.32775494e-03,  7.23329997e-03,
          4.47745960e-02,  1.51376078e-02,  8.56508087e-02, -9.53559495e-02,
         -1.42429495e-01,  1.65219698e-02,  1.44281567e-01,  7.00972259e-02,
         -9.14110661e-03,  1.34304548e-01,  1.28346872e-03,  2.91789327e-03,
         -3.16278797e-03,  2.97877420e-02, -2.39778413e-02,  8.00354455e-02,
         -3.65029405e-02, -1.68951529e-02,  3.84638021e-02, -8.71292806e-04,
         -1.87832261e-02, -1.29770380e-02, -5.28666333e-05,  6.38248020e-02,
          1.01629784e-03, -1.14842951e-03,  3.12715054e-04, -9.48729194e-03,
          2.42529714e-03, -3.41381178e-02,  5.65409044e-03,  1.70405996e-02,
         -5.42327571e-03,  1.50839969e-05,  1.42655314e-03,  9.45802810e-04,
          1.98180148e-03, -1.75616330e-01, -1.68301637e-04, -8.07381846e-06,
          1.27609631e-03, -8.26711681e-05,  5.65673776e-03, -2.90347512e-04,
         -2.73707801e-05,  4.01432643e-04, -5.61219511e-05, -2.30687540e-05,
          2.12279182e-04, -5.20734052e-02,  6.27240400e-06, -8.66319008e-05,
          3.40705615e-05, -3.88826242e-04, -7.19804693e-04, -1.47991326e-05,
          8.75708341e-07,  8.39300953e-02,  2.88533928e-06, -6.57683470e-06,
          1.99196652e-06,  4.43580619e-05,  2.13564992e-07,  1.99882202e-02,
         -3.68308253e-08,  4.32686504e-07,  9.85089960e-07, -1.32036159e-02,
         -9.50267208e-09, -3.32339019e-08, -2.76427804e-03])
expansion0= np.array([ 9., 10., 10.,  6.,  8.,  1.,  8.,  5.,  0.,  7.,  0.,  2.,  6.,
          6., 10.,  6.,  4.,  5.])


# model 1
weights1 = np.array([ 7.08013650e-01, -3.12830437e-01, -3.66878462e-01, -3.71533659e-02,
          7.72423009e-02,  2.48892312e-02,  2.28763446e+02, -2.42332863e-02,
          1.63780850e-02, -7.34270504e+01,  1.50592247e-03,  4.77961981e-03,
         -7.46890764e+01, -1.58007451e-02, -1.14193682e-02,  4.68782645e-03,
         -3.07682851e-03, -5.45116899e+03, -7.29261966e-03,  2.99528840e-03,
          5.29269170e+03, -4.47391184e-01, -1.17708065e-01, -3.50180066e-01,
          5.91656383e-02, -1.13716469e-01, -1.89517731e-02, -6.45981423e-02,
         -4.10113124e-03,  2.94762315e-01, -1.47358088e-02, -3.25377366e-02,
          8.28490500e-03, -1.01012288e-02, -6.21835620e-02,  5.34422733e-02,
         -1.95714661e-02, -3.28439901e+03,  1.71411702e-01,  1.29098316e-02,
          3.28432520e+03, -1.75013854e-01,  9.44496238e-02,  8.89815955e-02,
         -7.45926174e-03,  1.76759253e-04,  3.72792046e-03,  1.80989489e-02,
          1.83905663e-01, -1.87809553e-02, -2.68614111e-04,  5.02944769e-04,
          1.40704619e-02,  2.77141724e-02, -3.53935789e-03,  1.11491480e+03,
          3.82997055e-03, -3.26054067e-03, -1.11491562e+03,  1.51912188e-01,
          1.64047662e-02,  1.13632379e-01,  2.65630763e-04,  3.89786255e-02,
         -1.55172024e-04, -1.89982936e-03, -1.69189120e-01,  5.15376014e-03,
          2.03818513e-03, -2.50776211e-02,  1.98417326e-03, -7.41841566e+01,
         -1.21664572e-02,  7.41982185e+01, -3.62879997e-02, -1.58856746e-02,
         -6.28061903e-02,  1.00482261e-02,  6.82763977e-05, -1.26959856e-01,
         -5.29555930e-04, -3.62642601e-03, -9.56424600e-03, -2.45377077e-04,
         -9.49694278e-04, -4.65367923e-03,  3.98949524e-03,  3.18470205e-03,
          1.36537202e-02, -7.18634770e-03, -1.25762338e-02,  2.36715454e-05,
          3.39976969e-03,  1.64758997e-05, -2.08167471e-04,  6.37787307e-04,
         -2.08943189e-04, -2.63622222e-04, -1.56139223e-03, -7.04640335e-05,
         -3.85267137e-07, -6.69517915e-07, -3.99887335e-05,  4.21542364e-06,
          7.95878646e-06,  9.89375068e-05,  1.89703820e-04,  9.43831562e-07,
         -3.28673094e-06,  4.46819570e-08])
expansion1 =np.array([ 8.,  8., 10.,  4.,  8.,  4.,  5.,  2.,  6.,  7.,  3.,  2.,  3.,
          5.,  6.,  0.,  1.,  7.,  4.,  6.,  3.,  8.])


# model 2 
weights2 = np.array([ 8.95330686e-01, -1.56096511e-01, -3.99985034e-01,  1.74192723e-01,
          1.99917461e-01, -2.45835141e-02,  6.75197859e-02,  2.76874243e-02,
         -8.26631988e-02, -6.09030036e+02, -3.33265531e-01,  5.01106418e-02,
          1.64063958e-01,  1.38947278e+02,  8.78498840e-03, -1.81102425e-02,
          1.37161201e+02, -1.22332551e-02,  7.45314804e-04, -1.15698764e-01,
          2.16054468e-03, -1.01932200e-01,  1.07711039e-01, -2.96697243e-02,
          1.25382997e-01,  2.71330974e-03,  2.57420337e-03,  5.06781719e+02,
         -9.47458843e-01, -1.44326103e-01, -5.62883158e-04,  1.89883067e-01,
          1.62152451e-01,  3.40713455e-03, -1.23317203e-01,  9.30911173e-02,
         -9.84397068e-02,  1.31275366e-01,  9.55767555e-02,  3.89657490e-02,
          7.72843324e-02, -2.10354312e-02,  3.72725546e-02, -1.07859837e-01,
          4.22838004e-02,  1.49600903e-03, -2.71445698e-02,  1.52123584e-03,
          5.62823490e-02,  1.29594302e-03,  8.91455955e-02, -1.15122919e-02,
         -1.25727563e-02,  3.18640814e-02, -1.37906939e-01,  4.05297252e-02,
          4.96043571e-02, -3.60536296e-04, -4.30842421e-02,  7.42895969e-02,
          6.34355569e-03,  3.96655390e-02, -1.01476677e-01,  3.48550125e-01,
         -4.19291728e-02, -3.36408011e-02, -2.40642730e-02,  3.76565775e-02,
          3.22255982e-02,  1.23983910e-02, -1.59186520e-04,  3.52842813e-02,
          1.33143405e-04, -1.21805854e-03,  6.03506655e-03,  1.08912474e-01,
         -2.69105818e-03,  6.71737672e-04,  3.14079221e-01,  2.91351774e-02,
         -1.44439705e-02, -4.68971565e-02,  2.07491218e-04, -1.38821348e-02,
         -4.10478710e-02, -6.62859756e-03,  4.13685032e-02,  1.00979501e-01,
          9.34549066e-03,  5.69652995e-03, -2.46835368e-02, -2.74436457e-02,
         -6.91214731e-03, -1.00921856e-01,  1.96129165e-03, -1.76687948e-02,
         -1.41779498e-02, -1.84835703e-02, -1.16457417e-01,  5.87131384e-03,
         -1.09644882e-01, -1.41815147e-02,  1.87072166e-03,  8.92879078e-03,
          4.98522494e-03,  2.25649642e-02,  3.56016139e-04, -8.36675279e-03,
         -3.43750125e-01, -3.96106704e-04,  1.12176260e-02, -1.91145381e-02,
          8.29298541e-04, -8.60290051e-03,  1.07467325e-03,  4.80500976e-03,
          5.11101112e-03,  1.02107683e-02, -9.61056767e-02,  1.77616763e-02,
          2.47492749e-03, -1.29850278e-04,  6.65028855e-03, -9.29255549e-04,
         -5.60857639e-03,  8.74295210e-04, -2.70481184e-01,  9.37187489e-06,
          1.09558162e-02,  5.60437546e-03, -4.63618558e-05,  3.97123854e-02,
         -7.21683655e-04, -6.84525560e-04, -2.46546987e-03,  8.33552010e-02,
         -1.51501444e-03, -2.10520318e-04,  4.99001410e-06, -3.47477624e-03,
          7.16078280e-04, -4.52457048e-05, -5.56459700e-02, -1.46549825e-03,
          3.08496767e-03,  8.61053147e-07,  1.59569367e-03,  5.46436269e-05,
          1.82773217e-05,  2.98526989e-04,  2.71987433e-02,  6.58758249e-05,
          8.76341652e-06, -9.99770413e-08,  6.71257472e-04, -4.54333962e-05,
          9.15132693e-07, -1.37131538e-03, -4.84784757e-03, -1.60031421e-06,
          3.65002351e-06, -1.75084733e-05, -2.86358246e-02, -1.15176409e-06,
         -1.42738458e-07,  8.14754632e-10, -5.97782177e-05,  1.13005375e-06,
         -2.39246187e-07,  3.82819463e-07, -1.70228434e-03,  2.04258083e-06,
          3.54256480e-03])

expansion2 = np.array([ 9.,  9.,  9., 10.,  4.,  2.,  1.,  6.,  9.,  5.,  8.,  7.,  4.,
          6.,  8.,  7.,  7.,  8.,  5.,  8.,  3.,  9.,  9.,  0., 10.,  2.,
          1.,  4.,  3.])

# Means and Stds used in data preprocessing
tX_0_means = np.array(
[ 1.18304317e+02 , 5.87555243e+01,  8.19049079e+01,  1.38106587e+01,
  2.66512358e+00 , 1.38106586e+01,  7.63853200e+01,  1.39193395e+00,
 -9.09213762e-01 , 3.40278130e+01, -2.47520165e-02, -1.80050620e-02,
  4.23575064e+01 ,-5.26417979e-02,  4.57059910e-02,  3.15062965e+01,
 -2.63643545e-02 , 1.25898912e+02])
tX_1_means= np.array(
[ 1.21107723e+02,  4.60702098e+01 , 8.21586846e+01,  6.59084581e+01,
  2.33810440e+00 , 1.66403276e+01 , 1.50366979e+02,  1.44397495e+00,
  2.35147156e-01  ,3.85858806e+01 ,-1.49094317e-03, -7.48406112e-03,
  4.67455224e+01 , 5.10184284e-03 , 4.42591713e-02,  4.01139124e+01,
 -7.57626052e-03 , 2.03422270e+02 , 6.50355748e+01, -3.45231214e-03,
 -1.62293825e-02 , 6.50355750e+01])
tX_2_means= np.array(
[ 1.22320060e+02,  3.95757832e+01 , 7.91988681e+01,  1.09965938e+02,
  2.40332876e+00 , 3.71354811e+02 ,-8.21357727e-01,  2.00848348e+00,
  2.83457497e+01  ,2.79803358e+02 , 1.49478061e+00,  5.59890529e-01,
  4.58145792e-01 , 4.52708392e+01 ,-2.82943134e-03,  3.15430140e-03,
  5.25232269e+01, -3.70524202e-03 , 4.62548334e-02,  5.74216912e+01,
  7.90020392e-03,  3.32005117e+02 , 1.05831460e+02, -5.73158242e-03,
 -1.06631250e-02,  5.75806995e+01 ,-1.45229137e-02, -1.91196357e-03,
  1.82009296e+02])
tX_0_stds= np.array(
[44.8963225 , 31.98291989, 38.1266853 , 14.07914401,  0.69399154 ,14.07914402,
 23.55807989,  0.58105946,  0.93790322 ,15.2327093 ,  1.23426158 , 1.81773795,
 14.58382511,  1.31130608,  1.81615828 ,18.23585504,  1.81222145 ,53.06142103])
tX_1_stds= np.array(
[56.3060471,  35.52507745 ,42.28017737 ,47.43631384 , 0.73837075 ,16.98929712,
 65.1385198 ,  0.84592587 , 1.1066982  ,20.98616804 , 1.21662525 , 1.81386647,
 21.3018586 ,  1.26270422 , 1.8146682  ,28.39334354 , 1.81110307 ,81.87539317,
 45.17936237,  1.82000964 , 1.81789551 ,45.17936237])
tX_2_stds= np.array([ 59.11893084 , 36.31470897 , 43.06570745 , 75.56084413  , 1.7425907,
                     398.05229977 ,  3.58057832 ,  0.7850483  , 29.76201843 ,128.8705345,
                       1.10277207 ,  0.96834368 ,  0.39872434 , 29.42365161 ,  1.18307617,
                       1.81940289 , 28.94530846 ,  1.20035972 ,  1.81726774 , 43.61016349,
                       1.81535987 ,138.7155708  , 67.40582106 ,  1.74452033  , 1.8077286,
                      31.86983368 ,  2.03133074 ,  1.81769667 ,107.2811732 ])

#### 3. Data splitting, cleaninig, standardizing and feature_expansion

In [181]:
#splitting data by pri_jet_num
tX_split_test, y_split, idx_jet_num = split_dataset_by_jet_num(tX_test,np.ones(tX_test.shape[0]), remove_cols=True)
tX_0_test = tX_split_test[0]
tX_1_test = tX_split_test[1]
tX_2_test = tX_split_test[2]

In [144]:
#cleaning data
tX_0_test = clean_data(tX_0_test)
tX_1_test = clean_data(tX_1_test)
tX_2_test = clean_data(tX_2_test)

In [145]:
#means and std are learned on training data
tX_0_test, _ , _  = standardize_data(tX_0_test, tX_0_means, tX_0_stds)
tX_1_test, _ , _   = standardize_data(tX_1_test, tX_1_means, tX_1_stds)
tX_2_test, _ , _   = standardize_data(tX_2_test, tX_2_means, tX_2_stds)

In [146]:
#feature expansion
tX_0_test = polynomial_feature_expansion(tX_0_test, expansion0)
tX_1_test = polynomial_feature_expansion(tX_1_test, expansion1)
tX_2_test = polynomial_feature_expansion(tX_2_test, expansion2)

#### 4. Data predicting

In [ ]:
#predicting
y0_test = predict_labels(weights0,tX_0_test)
y1_test = predict_labels(weights1,tX_1_test)
y2_test = predict_labels(weights2,tX_2_test)

In [148]:
#connecting predictions and corresponding ids
y_pred = np.concatenate([y0_test, y1_test,y2_test],axis=0)
indices = np.concatenate(list(map(lambda x: x[0], idx_jet_num)),axis=0)
rearranged_ids = ids_test[indices]

#### 6. Write to file

In [149]:
OUTPUT_PATH = './least_squeares12.csv' # TODO: fill in desired name of output file for submission
create_csv_submission(rearranged_ids, y_pred, OUTPUT_PATH)